In [1]:
import requests

API_KEY = "xxxxx"
MPAN = "xxxxx"
SERIAL = "xxxxx"

In [5]:
url = f"https://api.octopus.energy/v1/electricity-meter-points/{MPAN}/meters/{SERIAL}/consumption/"
data = requests.get(url, auth=(API_KEY, ""))



<bound method Response.json of <Response [200]>>


In [7]:
print(data.json())


{'count': 48, 'next': None, 'previous': None, 'results': [{'consumption': 0.105, 'interval_start': '2020-11-22T23:00:00Z', 'interval_end': '2020-11-22T23:30:00Z'}, {'consumption': 0.152, 'interval_start': '2020-11-22T22:30:00Z', 'interval_end': '2020-11-22T23:00:00Z'}, {'consumption': 0.252, 'interval_start': '2020-11-22T22:00:00Z', 'interval_end': '2020-11-22T22:30:00Z'}, {'consumption': 0.383, 'interval_start': '2020-11-22T21:30:00Z', 'interval_end': '2020-11-22T22:00:00Z'}, {'consumption': 0.86, 'interval_start': '2020-11-22T21:00:00Z', 'interval_end': '2020-11-22T21:30:00Z'}, {'consumption': 0.185, 'interval_start': '2020-11-22T20:30:00Z', 'interval_end': '2020-11-22T21:00:00Z'}, {'consumption': 0.338, 'interval_start': '2020-11-22T20:00:00Z', 'interval_end': '2020-11-22T20:30:00Z'}, {'consumption': 0.207, 'interval_start': '2020-11-22T19:30:00Z', 'interval_end': '2020-11-22T20:00:00Z'}, {'consumption': 0.388, 'interval_start': '2020-11-22T19:00:00Z', 'interval_end': '2020-11-22T19

In [30]:
from apd.sensors.base import HistoricalSensor, JSONSensor
import typing as t
import datetime
import pint
from pint import _DEFAULT_REGISTRY as ureg


In [31]:
class PowerUsageSensor(HistoricalSensor[t.Any], JSONSensor[t.Any]):
    def historical(
        self, start: datetime.datetime, end: datetime.datetime
    ) -> t.Iterable[t.Tuple[datetime.datetime, t.Any]]:
        url = f"https://api.octopus.energy/v1/electricity-meter-points/{MPAN}/meters/{SERIAL}/consumption/"
        data = requests.get(url, params={"period_from": start.isoformat(), "period_to": end.isoformat(), "page_size": 25000}, auth=(API_KEY, ""))
        print(end.isoformat())
        results = data.json()["results"]
        for item in results:
            yield datetime.datetime.fromisoformat(item['interval_end'].strip("Z")), ureg.Quantity(item['consumption'], ureg.kilowatthour)


In [32]:
tuple(PowerUsageSensor().historical(datetime.datetime(2020,11,22,12,0,0), datetime.datetime(2020,11,22,23,0,0)))

2020-11-22T23:00:00


((datetime.datetime(2020, 11, 22, 23, 30), 0.105 <Unit('kilowatt_hour')>),
 (datetime.datetime(2020, 11, 22, 23, 0), 0.152 <Unit('kilowatt_hour')>),
 (datetime.datetime(2020, 11, 22, 22, 30), 0.252 <Unit('kilowatt_hour')>),
 (datetime.datetime(2020, 11, 22, 22, 0), 0.383 <Unit('kilowatt_hour')>),
 (datetime.datetime(2020, 11, 22, 21, 30), 0.86 <Unit('kilowatt_hour')>),
 (datetime.datetime(2020, 11, 22, 21, 0), 0.185 <Unit('kilowatt_hour')>),
 (datetime.datetime(2020, 11, 22, 20, 30), 0.338 <Unit('kilowatt_hour')>),
 (datetime.datetime(2020, 11, 22, 20, 0), 0.207 <Unit('kilowatt_hour')>),
 (datetime.datetime(2020, 11, 22, 19, 30), 0.388 <Unit('kilowatt_hour')>),
 (datetime.datetime(2020, 11, 22, 19, 0), 0.727 <Unit('kilowatt_hour')>),
 (datetime.datetime(2020, 11, 22, 18, 30), 0.501 <Unit('kilowatt_hour')>),
 (datetime.datetime(2020, 11, 22, 18, 0), 0.171 <Unit('kilowatt_hour')>),
 (datetime.datetime(2020, 11, 22, 17, 30), 0.254 <Unit('kilowatt_hour')>),
 (datetime.datetime(2020, 11, 22